In [1]:
import ipywidgets as widgets
widgets.HTML(value="Hello <b>World</b>")

Widget Javascript not detected.  It may not be installed properly.


In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import os
import re
import math
import random

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
data_files = os.listdir('data/')
DATA_PATH = 'data/'
print(len(data_files))

## Expected output: 221

221


In [4]:
## Load each song, do some basic preprocessing, and store each in 

songs = []

for file in data_files:
    with open(DATA_PATH+file, 'r') as f:
        joined_lyrics = ''
        raw_data = f.readlines()[2:]
        joined_lyrics = "".join(raw_data)
        songs.append(re.sub(r' *\[.*?\]', '', joined_lyrics))

combined_lyrics = "\n".join(songs)

Data preprocessing code borrowed from this colab notebook: https://colab.research.google.com/drive/1n45bHMFw5dSTEUxcGYrQxhApPaltqGsS#scrollTo=6CED-DjGO7Ce

In [5]:
def create_lyric_pairs(lyrics):
    ''' Function that creates pairs of lyrics
        Args: lyrics - multiple songs by an artist in a text file
        Returns: 2 lists - source and target sentences
    '''
    source = []
    target = []
    
    lyrics_ = re.sub(r' \([^)]*\)', '', lyrics)
    lyrics_ = lyrics_.replace('(Hook)', '')

    lyrics_ = lyrics_.split('\n')

    # Clean up extra spaces in the lyrics
    counter = 0
    for i in range(len(lyrics_)-1):
        if lyrics_[counter] == "" and lyrics[counter+1] == "":
            lyrics_[counter+1].pop()
        counter += 1
    
    counter = 0
    for i in range(len(lyrics_)-1):
        if lyrics_[counter] == "" and lyrics[counter+1] == "":
            lyrics_[counter+1].pop()
        counter += 1    
    # replace a "" with the word break
    for i in range(len(lyrics_)):
        if lyrics_[i] == "":
            lyrics_[i] = "break"
    
    counter = 0
    for i in range(len(lyrics_) - 1):
        if lyrics_[counter] != "break" and lyrics_[counter+1] != "break":
            # Use one line and target is the next line
            source.append(lyrics_[counter])
            target.append(lyrics_[counter+1])
            
        counter += 1
            
    return source, target

def noise_sentence(sentence_, percent_words, replacement_token = "<mask>"):
    '''
    Args: sentence - the sentence to noise
          percent_words - the percent of words to remove
    '''
    # Create a list item and copy
    sentence_ = sentence_.split(' ')
    sentence = sentence_.copy()
    
    num_words = math.ceil(len(sentence) * percent_words)
    
    # Create an array of tokens to sample from; don't include the last word as an option because in the case of lyrics
    # that word is often a rhyming word and plays an important role in song construction
    sample_tokens = set(np.arange(0, np.maximum(1, len(sentence)-1)))
    
    words_to_noise = random.sample(sample_tokens, num_words)
    
    # Remove redundant spaces
    sentence = re.sub(r' {2,5}', ' ', ' '.join(sentence))
    
    # Combine concurrent <mask> tokens into a single token; this just does two rounds of this; more could be done
    sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
    sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
    return sentence

In [6]:
source_lyrics, target_lyrics = create_lyric_pairs(combined_lyrics)

In [7]:
lyrics_df = pd.DataFrame({'source':source_lyrics, "target":target_lyrics})

NOTE: Leaving this here because it was a problem for person who wrote the data cleaning code we're using here. We may or may not need it, depending on if there are colons in the dataset. 

```
# Clean out the lyrics that contain a ":"; this was specific to some lyrics and created some 
lyrics_df = lyrics_df[~lyrics_df.source.str.contains(":")]
lyrics_df = lyrics_df[~lyrics_df.target.str.contains(":")]
lyrics_df = lyrics_df[lyrics_df.target != "break"]
```

In [8]:
lyrics_df['source'] = lyrics_df['source'].apply(lambda x: noise_sentence(x, 0.25))

This visualization code is broken, figure out how to visualize the length of each sentence when there is time

In [9]:
# # Out of interest take a look at the distribution of sentence lengths
# # It turns out there was something a bit funny going on here, but as the as the tokenizer limits the length of the sentences
# # fed to the model, it's not really an issue (aside from the document with training data possibly being a bit larger than it needs to be)
# import matplotlib.pyplot as plt

# length_of_sent = []
# for i in range(500000):
#     length_of_sent.append(len(lyrics_df.iloc[i, 1].split(' ')))
    
# plt.hist(length_of_sent)

# Modeling

TODO: build the model!



In [10]:
lyrics_df.loc[0]

source               This is not your ordinary ballad
target    With a perfect little bow around the middle
Name: 0, dtype: object

In [11]:
lyrics_df.loc[1]

source    With a perfect little bow around the middle
target            And a black knight on a white horse
Name: 1, dtype: object

In [12]:
# Load the model
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained(
    "facebook/bart-base")


Widget Javascript not detected.  It may not be installed or enabled properly.


AttributeError: 'FloatProgress' object has no attribute 'style'